In [1]:
import orjson
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report

In [2]:
def find_last(text: str, label_words: list[str]) -> str:
    for word in reversed(text.replace(':', ' ').split()):
        if word.lower() in label_words:
            return word.lower()

In [3]:
with open('../training/test.json', 'rb') as f:
    GT = orjson.loads(f.read())

In [4]:
GT = {sample['image']: find_last(sample['conversations'][-1]['value'], ['yes', 'no']) for sample in GT}
GT_label = {k: 1 if v == 'yes' else 0 for k, v in GT.items()}

In [5]:
# print true images
for image in GT:
    if GT[image] == 'yes':
        print(image)

ig_bukittimahpoly/ig_bukittimahpoly/2021-05-20_01-21-04_UTC.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-29_13-43-14_UTC.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-24_00-59-08_UTC_1.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-06-09_11-52-54_UTC.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-27_13-06-45_UTC_2.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-23_01-05-41_UTC_4.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-06-04_02-38-56_UTC_1.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-06-07_10-24-57_UTC.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-23_01-05-41_UTC_2.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-17_07-10-26_UTC.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-07-11_11-47-50_UTC.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-27_13-06-45_UTC_3.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-23_01-05-41_UTC_3.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-07-16_10-12-39_UTC.png
ig_bukittimahpoly/ig_bukittimahpoly/2021-05-28_05-05-02_UTC.png
ig_bukittimahpoly/ig_bukit

In [11]:
with open('llava_pipeline_output_with_sampling_retry.json', 'rb') as f:
    output = orjson.loads(f.read())
    output = {k.replace('../scraping/', ''): v['score'] for k, v in output.items()}

label_list = []
score_list = []
for image in GT_label:
    label_list.append(GT_label[image])
    score_list.append(output[image] if image in output else 0.5)  # some images are not in the output

print(accuracy_score(label_list, [1 if score > 0.5 else 0 for score in score_list]))
print(roc_auc_score(label_list, score_list))
print(classification_report(label_list, [1 if score > 0.5 else 0 for score in score_list]))

0.8063513979979289
0.7990746074660715
              precision    recall  f1-score   support

           0       0.83      0.93      0.87      2116
           1       0.71      0.48      0.57       781

    accuracy                           0.81      2897
   macro avg       0.77      0.70      0.72      2897
weighted avg       0.80      0.81      0.79      2897



In [14]:
import glob
fnames = glob.glob('qwen_pipeline_output*.json')
for fname in fnames:
    print(fname)
    with open(fname, 'rb') as f:
        output = orjson.loads(f.read())
        output = {k.replace('../scraping/', ''): v['score'] for k, v in output.items()}
    
    label_list = []
    score_list = []
    for image in GT_label:
        label_list.append(GT_label[image])
        score_list.append(output[image] if image in output else 0.5)  # some images are not in the output
    
    print(accuracy_score(label_list, [1 if score > 0.5 else 0 for score in score_list]))
    print(roc_auc_score(label_list, score_list))
    print(classification_report(label_list, [1 if score > 0.5 else 0 for score in score_list]))

qwen_pipeline_output_Qwen2-VL-7B-Instruct.json
0.6969278564031757
0.6439656153106991
              precision    recall  f1-score   support

           0       0.77      0.84      0.80      2116
           1       0.42      0.32      0.36       781

    accuracy                           0.70      2897
   macro avg       0.59      0.58      0.58      2897
weighted avg       0.67      0.70      0.68      2897

qwen_pipeline_output_qwen2-vl-7b-pissa-r128-a256-lr1e4-with-sg-no-eval-sampling-merged.json
0.794269934414912
0.8025119266898866
              precision    recall  f1-score   support

           0       0.82      0.92      0.87      2116
           1       0.68      0.44      0.54       781

    accuracy                           0.79      2897
   macro avg       0.75      0.68      0.70      2897
weighted avg       0.78      0.79      0.78      2897

qwen_pipeline_output_qwen2-vl-7b-pissa-r128-a256-lr1e4-without-sg-memes-merged.json
0.7787366240938902
0.7499588526173365
          